# The Model

## reloading
sometimes it helps to reload data -- redo imports up here

In [1]:
from importlib import reload

In [21]:
import neural_net as nen
reload(nen)
from neural_net import *

## setup
Import, load data and instantiate relevant classes (DataLoader, Dataset etc)

In [3]:
## imports
#  modules
import numpy as np
import pandas as pd
import xarray as xr
import warnings
import os

import torch
from torch import nn

# custom code
from dask_addons import FlattenedDaskDataset
from neural_net import DNN, geoDataset, BatchDataLoader, train, test
from open_fire_data import FlattenedTruthTable

warnings.filterwarnings('ignore')

In [ ]:
from dask.distributed import Client
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 15.52 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:50097,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:50118,Total threads: 2
Dashboard: http://127.0.0.1:50122/status,Memory: 3.88 GiB
Nanny: tcp://127.0.0.1:50100,


2025-08-07 14:34:40,801 - distributed.scheduler - WARNING - Detected different `run_spec` for key ('rechunk-split-d13740ca114312c72ee01462b8c29945', 48) between two consecutive calls to `update_graph`. This can cause failures and deadlocks down the line. Please ensure unique key names. If you are using a standard dask collections, consider releasing all the data before resubmitting another computation. More details and help can be found at https://github.com/dask/dask/issues/9888. 
Debugging information
---------------------
old task state: released
old run_spec: Alias(('rechunk-split-d13740ca114312c72ee01462b8c29945', 48)->('getitem-rechunk-split-d13740ca114312c72ee01462b8c29945', 48))
new run_spec: <Task ('rechunk-split-d13740ca114312c72ee01462b8c29945', 48) getitem(...)>
old dependencies: {('getitem-rechunk-split-d13740ca114312c72ee01462b8c29945', 48)}
new dependencies: frozenset({('getitem-a3780eb038d725ad75047040c2951fb2', 0, 1, 1)})

2025-08-07 14:34:41,801 - distributed.schedule

In [5]:
DATA_DIR_PATH = '../data/_ZARR_FILES'

data_path = os.path.join(DATA_DIR_PATH, 'alaska_full.zarr')
prior_data_path = os.path.join(DATA_DIR_PATH, 'alaska_prior.zarr')

data = xr.open_zarr(data_path, decode_timedelta=False)
prior_data = xr.open_zarr(prior_data_path, decode_timedelta=False)

In [6]:
input_ds = FlattenedDaskDataset(data, prior_data)
input_ds.setup()

[compute_running_totals] run time: 0.1680610179901123s


In [7]:
fire_data = FlattenedTruthTable(
    pd.read_csv('../data/_FIRE/alaska_range_csv/data.csv'),
    lat_vals = input_ds.latitude,
    long_vals = input_ds.longitude
)

In [8]:
ds = geoDataset(input_ds, fire_data, feature_num = 13)

## training


In [9]:
model = DNN()
print(model)

DNN(
  (network): Sequential(
    (0): Linear(in_features=13, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=10, bias=True)
    (3): ReLU()
    (4): Linear(in_features=10, out_features=1, bias=True)
    (5): Sigmoid()
  )
)


In [ ]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [22]:
train_dataloader = BatchDataLoader(ds, end=100)
test_dataloader = BatchDataLoader(ds)

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [23]:
len(train_dataloader)

3328800

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer, device)
    test(test_dataloader, model, loss_fn, device)
print("Done!")

Epoch 1
-------------------------------
loss:     nan  [   65/3328800]
loss:     nan  [  715/3328800]
loss:     nan  [ 1365/3328800]
loss:     nan  [ 2015/3328800]
loss:     nan  [ 2665/3328800]
loss:     nan  [ 3315/3328800]
loss:     nan  [ 3965/3328800]
loss:     nan  [ 4615/3328800]
loss:     nan  [ 5265/3328800]
loss:     nan  [ 5915/3328800]
loss:     nan  [ 6565/3328800]
loss:     nan  [ 7215/3328800]
loss:     nan  [ 7865/3328800]
loss:     nan  [ 8515/3328800]
loss:     nan  [ 9165/3328800]
loss:     nan  [ 9815/3328800]
loss:     nan  [10465/3328800]
loss:     nan  [11115/3328800]
loss:     nan  [11765/3328800]
loss:     nan  [12415/3328800]
loss:     nan  [13065/3328800]
loss:     nan  [13715/3328800]
loss:     nan  [14365/3328800]
loss:     nan  [15015/3328800]
loss:     nan  [15665/3328800]
loss:     nan  [16315/3328800]
loss:     nan  [16965/3328800]
loss:     nan  [17615/3328800]
loss:     nan  [18265/3328800]
loss:     nan  [18915/3328800]
loss:     nan  [19565/3328800]